# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [14]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
# Feature Scaling/Selection
# Drop rows containing missing values
spaceship = spaceship.dropna()

# Cabin is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])

# Drop PassengerId and Name
spaceship = spaceship.drop(['PassengerId', 'Name'], axis=1)

# For non-numerical columns, do dummies.
# Asegurarse de que las columnas booleanas estén correctamente tipificadas como bool
spaceship['Transported'] = spaceship['Transported'].astype(bool)
# Seleccionar solo columnas no numéricas
non_numeric_columns = spaceship.select_dtypes(include=['object']).columns

# Convertir columnas no numéricas en dummies
spaceship = pd.get_dummies(spaceship, columns=non_numeric_columns)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [23]:
# Train Test Split
features = spaceship.drop(columns=['Transported'])
target = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)

X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [24]:
# Bagging and Pasting
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

- Evaluate your model

In [25]:
# Evaluate your model
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.26843250163033916
RMSE 0.36570080168752395
R2 score 0.464973324103235


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [26]:
# Check best hyperparameters
model = BaggingRegressor(DecisionTreeRegressor())
params = model.get_params()

# Imprimir los hiperparámetros
for param, value in params.items():
    print(f"{param}: {value}")

base_estimator: deprecated
bootstrap: True
bootstrap_features: False
estimator__ccp_alpha: 0.0
estimator__criterion: squared_error
estimator__max_depth: None
estimator__max_features: None
estimator__max_leaf_nodes: None
estimator__min_impurity_decrease: 0.0
estimator__min_samples_leaf: 1
estimator__min_samples_split: 2
estimator__min_weight_fraction_leaf: 0.0
estimator__random_state: None
estimator__splitter: best
estimator: DecisionTreeRegressor()
max_features: 1.0
max_samples: 1.0
n_estimators: 10
n_jobs: None
oob_score: False
random_state: None
verbose: 0
warm_start: False


In [27]:
# Define hyperparameters to fine tune
grid = {
    "n_estimators": [10, 50, 100, 200], 
    "estimator__max_leaf_nodes": [None, 10, 20, 50, 100],  
    "estimator__max_depth": [None, 5, 10, 15, 20] 
}


In [28]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor())

- Run Grid Search

In [29]:
model = GridSearchCV(estimator = bagging_reg, param_grid = grid, cv=5)

In [30]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [None, 5, 10, 15, 20],
                         'estimator__max_leaf_nodes': [None, 10, 20, 50, 100],
                         'n_estimators': [10, 50, 100, 200]})

In [32]:
best_model = model.best_estimator_
best_model

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=10,
                                                 max_leaf_nodes=50),
                 n_estimators=100)

- Evaluate your model

In [34]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.26906221647270356
RMSE 0.36545396768530347
R2 score 0.4656953252935261


In [35]:
# Print best hyperparameters
print("Best hyperparameters:")
print(model.best_params_)

Best hyperparameters:
{'estimator__max_depth': 10, 'estimator__max_leaf_nodes': 50, 'n_estimators': 100}
